In [314]:
import pandas as pd
from textblob import TextBlob
import re

In [315]:
df = pd.read_csv('tweets.csv', encoding="utf-8")

In [316]:
df.isnull().sum()

username             0
to                1874
text                93
retweets             0
favorites            0
replies              0
id                   0
permalink            0
author_id            0
date                 0
formatted_date       0
hashtags          8192
mentions          7833
geo               8205
urls              7272
dtype: int64

In [317]:
# drop insignificant columns
df.drop(['username', 'id', 'permalink', 'author_id', 'geo', 'hashtags', 'formatted_date', 'mentions'], axis=1, inplace=True)

In [318]:
df['text'].astype(str)

0                          @TheEconomist Interesting name
1       @Benzinga I didn't take anyone's car and never...
2       My best guess for 2016: ~70% landing success r...
3                          @JeffBezos @SpaceX Thanks Jeff
4       Falcon lands on droneship, but the lockout col...
                              ...                        
8200    Yeah, very important to provide C/BiPAP device...
8201    Invasive ventilators are for worst case patien...
8202    Exactly. Moreover, all hospitals were given ex...
8203                         Will call when we reach Mars
8204                                              Exactly
Name: text, Length: 8205, dtype: object

In [319]:
df.describe()

,retweets,favorites,replies
count,8205.000000,8.205000e+03,8205.000000
mean,2252.576843,1.807881e+04,487.715661
std,10393.932164,5.455258e+04,1594.076078
min,0.000000,6.000000e+01,0.000000
25%,41.000000,9.170000e+02,41.000000
50%,122.000000,2.339000e+03,98.000000
75%,982.000000,1.207400e+04,379.000000
max,391852.000000,1.712683e+06,51016.000000


In [320]:
df.head()

,to,text,retweets,favorites,replies,date,urls
0,TheEconomist,@TheEconomist Interesting name,263,1262,71,2016-01-23 19:43:17+00:00,NaN
1,Benzinga,@Benzinga I didn't take anyone's car and never...,47,242,53,2016-01-20 17:56:05+00:00,NaN
2,NaN,My best guess for 2016: ~70% landing success r...,1431,5151,393,2016-01-19 04:11:54+00:00,NaN
3,JeffBezos,@JeffBezos @SpaceX Thanks Jeff,158,1068,50,2016-01-18 03:09:21+00:00,NaN
4,NaN,"Falcon lands on droneship, but the lockout col...",8026,8693,1234,2016-01-18 03:07:21+00:00,https://www.instagram.com/p/BAqirNbwEc0/


In [321]:
# # parse date as index and convert timezone to EST
df['date'] = pd.to_datetime( df['date'], format='%Y-%m-%d %H:%M:%S')
df['date'] = df['date'].dt.tz_convert('EST')

In [322]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8205 entries, 0 to 8204
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   to         6331 non-null   object             
 1   text       8112 non-null   object             
 2   retweets   8205 non-null   int64              
 3   favorites  8205 non-null   int64              
 4   replies    8205 non-null   int64              
 5   date       8205 non-null   datetime64[ns, EST]
 6   urls       933 non-null    object             
dtypes: datetime64[ns, EST](1), int64(3), object(3)
memory usage: 448.8+ KB


In [323]:
# sentiment_analysis
def sentiment_calc(text):
    try:
        return TextBlob(text).sentiment
    except:
        return (0, 0)

df['sentiment'] = df['text'].apply(sentiment_calc)
df['polarity'] = df['sentiment'].apply(lambda x: tuple(x)[0])
df['subjectivity'] = df['sentiment'].apply(lambda x: tuple(x)[1])
df.drop(['sentiment'], axis=1, inplace=True)

In [324]:
# extract info of if url is news
reg = r'https?:\/\/([\w\d.]+)\/[\w\d\/\-\?\=]*'
news_keywords = r'wsj|news|forbes|bloomberg|finance|money|investopedia|marketwatch|cnbc|times|fortune|nasdaq|cnn|huffpost|cnn|usatoday|npr'
def get_domain(url):
    url = str(url)
    result =  re.search(reg, url)
    if result:
        return result.group(1)
    return None

df['url_domain'] = df['urls'].apply(get_domain)
df['url_news'] = df['url_domain'].str.contains(news_keywords)
df['url_news'].fillna(False, inplace=True)

df.drop(['url_domain'], axis=1, inplace=True)

In [325]:
# extract info of if url is video
df['url_video'] = df['urls'].str.contains('video|youtube.com|watch')
df['url_video'].fillna(False, inplace=True)
df.drop(['urls'], axis=1, inplace=True)

In [326]:
# find if tweet is a retweet
df['is_retweet'] = df['to'].notnull()
df.drop(['to'], axis=1, inplace=True)

In [327]:
# extract keyword count
tesla = '[Tt]esla'
closely_related_keywords = r'Model [\w\d]|Cars? | cars? |[Tt]ruck|[Ee]lectric|[Ss]olar|[Rr]oof|Semi|[Aa]uto(nomous|pilot|steer(ing)?)?|[Pp]anel|[Ee]nergy|[Mm]egapack|drive(ing)|[Tt]axi? | acceleration|top speed|brak(e|ing)'
money_related_keywords = r'[Aa]ffordable|[Cc]heap|[Ee]xpensive|[Pp]rice]|[Ss]tock'
other_related_keywords = r'[Ss]pace|[Mm]ars|[Ll]aunch|AI|neural|Boring Company|Hyperloop|[Ii]terview|boringcompany'
def word_match_count(text, pattern):
    if text is None:
        return 0
    text = str(text)
    return len(re.findall(pattern, text))

df['closely_related'] = df['text'].apply(word_match_count, args=(closely_related_keywords,))
df['money_related'] = df['text'].apply(word_match_count, args=(money_related_keywords,))
df['other_related'] = df['text'].apply(word_match_count, args=(other_related_keywords,))
df['tesla'] = df['text'].apply(word_match_count, args=(tesla,))
df.drop(['text'], axis=1, inplace=True)

In [328]:
# convert bool to int
df[['is_retweet', 'url_news', 'url_video']] = df[['is_retweet', 'url_news', 'url_video']].astype(int)

In [329]:
df.describe()

,retweets,favorites,replies,polarity,subjectivity,url_news,url_video,is_retweet,closely_related,money_related,other_related,tesla
count,8205.000000,8.205000e+03,8205.000000,8205.000000,8205.000000,8205.000000,8205.000000,8205.000000,8205.000000,8205.000000,8205.000000,8205.000000
mean,2252.576843,1.807881e+04,487.715661,0.119454,0.334493,0.007313,0.015235,0.771603,0.160024,0.006216,0.090189,0.101523
std,10393.932164,5.455258e+04,1594.076078,0.266830,0.312552,0.085206,0.122492,0.419826,0.509405,0.083122,0.340157,0.343575
min,0.000000,6.000000e+01,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,41.000000,9.170000e+02,41.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,122.000000,2.339000e+03,98.000000,0.000000,0.325000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,982.000000,1.207400e+04,379.000000,0.250000,0.566667,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,391852.000000,1.712683e+06,51016.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000,2.000000,4.000000,3.000000


In [330]:
df['day_of_week'] = df['date'].dt.dayofweek

In [331]:
def check_if_during_market(date):
    closing_time = dateteime(x.year, x.month, x.day, 16, 0, tzinfo=date.tzinfo)
    if date >= closing_time:
        date = date + timedelta(days=1)

df['date'] = df['date'].apply(lambda x: datetime(x.year, x.month, x.day, tzinfo=x.tzinfo))

In [332]:
from datetime import timedelta 
from datetime import datetime
def date_alt(df):
    date = df['date']
    dayofweek = df['day_of_week']
    if (dayofweek == 6):
        return date + timedelta(days=1)
    if (dayofweek == 5):
        return date + timedelta(days=2)
    return date

df['date'] = df[['date', 'day_of_week']].apply(date_alt, axis=1) 

In [333]:
sums_by_date = df[['date', 'retweets', 'favorites', 'replies', 'is_retweet', 'closely_related', 'money_related', 'other_related', 'tesla']].groupby(['date']).sum()
means_by_date = df[['date', 'polarity', 'subjectivity']].groupby(['date']).mean()
df2 = pd.merge(sums_by_date, means_by_date, on='date')

In [334]:
df2

,retweets,favorites,replies,is_retweet,closely_related,money_related,other_related,tesla,polarity,subjectivity
date,,,,,,,,,,
2016-01-04 00:00:00-05:00,871,3013,261,0,0,0,0,0,0.200000,0.150000
2016-01-11 00:00:00-05:00,12040,23921,1333,2,2,0,1,1,0.092000,0.473000
2016-01-12 00:00:00-05:00,856,2098,124,1,0,0,1,0,0.000000,0.000000
2016-01-15 00:00:00-05:00,1559,5520,322,1,0,0,0,0,0.425000,0.400000
2016-01-18 00:00:00-05:00,22010,43225,3949,6,0,0,2,0,0.130705,0.290235
...,...,...,...,...,...,...,...,...,...,...
2020-04-27 00:00:00-05:00,81131,738373,22426,43,2,0,3,0,0.147439,0.298169
2020-04-28 00:00:00-05:00,1119,21631,322,1,0,0,0,0,0.000000,0.000000
2020-04-29 00:00:00-05:00,137456,781791,77666,8,0,0,0,0,0.082341,0.175397


In [335]:
df2.to_pickle('tweets_processed.pkl')